In [1]:

import binascii
%matplotlib inline
from PIL import Image
import matplotlib

import numpy as np
import matplotlib.pyplot as plt
from skimage import data

import pickle
from scipy import ndimage
import cv2
import skimage.transform as   transform

from skimage.draw import line_aa, polygon
import pandas
import os

#im = sk.io.imread(pth)



In [419]:
pth = 'data/resize-3.tif'
#"data/----16019 Parcel U JAMAICA PLAIN, MA (community blders)------Superseded Drawings--Architectural--A1.01-FIRST-FLOOR-PLAN-Rev.0(1).tif"

img = Image.open(pth)
scale = 1
im2 = cv2.imread(pth)
#im2 = transform.rescale(im, scale)

In [2]:
def plot_image(im):
    fig, ax = plt.subplots(figsize=(18, 20))
    ax.imshow(im)


In [19]:
import csv
import re
"""
Import CSV data from OST SQL, and preprocess some data

"""


datapath = 'data/ddatar.csv'
dpi = 216


def unhex (lines, scale_mx):
    for h in range(0, len(lines)):
        binary_string = binascii.unhexlify(lines[h]['Position'][2:])
        x = np.array(binary_string.decode().split(';'))
        y = x.astype(np.float32) 
        scale = scale_mx / (int(lines[h]['page_Width']) * dpi)
        s = np.rint(y) * scale * 2.25
        lines[h]['coords'] = s.astype(int)
    return lines

def read_data (pth):
    lines = np.genfromtxt(pth, delimiter="," ,autostrip=True, dtype='str')
    my_dict = []
    for i in range(len(lines)):
        row = dict()
        for j in range(len(lines[0])):
            row[lines[0][j]] = lines[i][j]
        my_dict.append(row) 
    return np.array(my_dict)[1:]


def assign_color(lines, cat_label):
    colors = np.random.randint(0, high=190, size=(len(lines), 3)) 
    seen = []
    
    for h in range(0, len(lines)):
        if lines[h][cat_label] not in seen:
            seen.append(lines[h][cat_label])
        
        lines[h]['color'] = colors[seen.index(lines[h][cat_label])]
    return lines
        
def unique_cats(lines, cat_label):
    seen = []
    for h in lines:
        if h[cat_label] not in seen:
            seen.append( h[cat_label])
    return seen



# data = read_data(datapath)       
# res = unhex(data, scale_max)

# fnl = assign_color(res, 'catName')
# sn = unique_cats(res, 'catName')


In [20]:

"""
"""

def img_dir_to_name(line):
    return line.replace('G:\\Projects\\2016\\Precon', '').replace('\\' , '--')


def set_image_paths (line, path, img_dir):
    searches = line['ImagePath'].replace('tif' , 'jpeg').split('\\')
    s1 = searches[-1:][0]
   
    xs1 = re.findall('\d{5,6}', line['ImagePath'].replace('\\' , ' '))
    
    if xs1:
        s2 = xs1[0]
        tmp = [x for x in img_dir if s1 in x.replace(',', '') and s2 in x]
        if tmp:
            line['img_name'] = tmp[0]

    return line
    

In [21]:
im2pth = fnl_msk['img_loc']
#im2 = Image.open(im2pth)
im2 = cv2.imread(im2pth)
plot_image(im2)
fnl_msk

NameError: name 'fnl_msk' is not defined

In [13]:
scaled_path = 'scaled/'
masks_path = 'masks/'
dpi = 216

def set_colors(image, rr, cc, val, line):
    image[rr, cc, 0] = val * line['color'][0]
    image[rr, cc, 1] = val * line['color'][1]
    image[rr, cc, 2] = val * line['color'][2]
    return image

def draw_image_mask(image, lines, color_key, cats=[]):
    
    if cats:
        lines = [x for x in lines if any(x[color_key] in s for s in cats)]
        
    for line in lines:
        vec = line['coords']
        if vec.size == 5:
            rr, cc, val = line_aa(vec[1] , vec[0] , vec[3], vec[2])
            image = set_colors(image, rr, cc, val, line)
        else:
            a = vec[::2] #np.append(vec[:-1:2], vec[0])
            b = vec[1::2] #np.append(vec[1:-1:2], vec[1])
            rr, cc = polygon(b[::-1], a[::-1])
            image = set_colors(image, rr, cc, 1, line)
            
    return image


def process_image (single_dwg_data, uid, imgs_path, cats=[], scale_mx=2048, mode='make-mask', color_key='catName'): 
    single_dwg_data = unhex(single_dwg_data, scale_mx)

    #link image in folder
    img_dir = os.listdir(imgs_path + scaled_path)
    
    #scale = int(scale_mx / int(single_dwg_data[0]['page_Width']) * dpi )
   
    if mode == 'make-mask':
        
        
        img_location = set_image_paths(single_dwg_data[0], imgs_path, img_dir)
        
        if 'img_name' in img_location:
            
            im2 = cv2.imread(imgs_path + scaled_path + img_location['img_name'])
            
            nm = np.zeros(im2.shape) 
            nm.fill(255)
            img = draw_image_mask(nm, single_dwg_data, color_key, cats=cats)
            
            #img_w =  int(single_dwg_data[0]['page_Height'] * dpi )  #int(scale_mx * int(single_dwg_data[0]['page_Width'])  / int(single_dwg_data[0]['page_Height']))
            #img_h =  int(single_dwg_data[0]['page_Width'] * dpi )
    
            cv2.imwrite(imgs_path + masks_path + img_location['img_name'] , img)
            print("found + complete : " + single_dwg_data[0]['ImagePath'])
        else: 
            print("Image not found for " + single_dwg_data[0]['ImagePath'])
        

    return single_dwg_data
    

def process_images(imgs_path, cats=[], scale_mx=2048, mode='make-mask', color_key='catName'):

    data = read_data(datapath)
    data = assign_color(data, color_key)
    
    #segment sheet into drawings
    pgs = unique_cats(data, 'pgid')
    all_dwg_data = []
    
    for pgid in pgs:
        single_dwg_data = [x for x in data if x['pgid'] == pgid] 
        single_dwg_data= process_image(single_dwg_data, pgid, imgs_path, cats=cats,
                                       scale_mx=scale_mx, mode=mode, color_key = color_key)
        all_dwg_data = all_dwg_data + single_dwg_data
    
    return all_dwg_data


#plot_image(mod_img[0:256, 256:512])

#mod_img.shape[0]/256


In [14]:
im2 = cv2.imread('/home/psavine/data/sets/16019/imgs/scaled/----16019 Parcel U JAMAICA PLAIN, MA (community blders)------Superseded Drawings--Architectural--A1.04-FOURTH-FLOOR-PLAN-Rev.0(1).jpeg')
im2.shape

(183, 256, 3)

In [22]:
some_dt = process_images('../../data/sets/16019/imgs/', ['000 GSF'], scale_mx=256, color_key='catName' )

found + complete : G:\Projects\2016\Precon\16019 Parcel U JAMAICA PLAIN MA (community blders)\00 PROJECT DOCUMENTS\001 MOST CURRENT DWGS\Architectural\Converted\A1.00-BASEMENT-GARAGE-FLOOR-PLAN-Rev.0(1).tif
found + complete : G:\Projects\2016\Precon\16019 Parcel U JAMAICA PLAIN MA (community blders)\00 PROJECT DOCUMENTS\001 MOST CURRENT DWGS\Architectural\Converted\A1.01-FIRST-FLOOR-PLAN-Rev.0(1).tif
found + complete : G:\Projects\2016\Precon\16019 Parcel U JAMAICA PLAIN MA (community blders)\00 PROJECT DOCUMENTS\001 MOST CURRENT DWGS\Architectural\Converted\A1.02-SECOND-FLOOR-PLAN-Rev.0(1).tif
found + complete : G:\Projects\2016\Precon\16019 Parcel U JAMAICA PLAIN MA (community blders)\00 PROJECT DOCUMENTS\001 MOST CURRENT DWGS\Architectural\Converted\A1.03-THIRD-FLOOR-PLAN-Rev.0(1).tif
found + complete : G:\Projects\2016\Precon\16019 Parcel U JAMAICA PLAIN MA (community blders)\00 PROJECT DOCUMENTS\001 MOST CURRENT DWGS\Architectural\Converted\A1.04-FOURTH-FLOOR-PLAN-Rev.0(1).tif
fou

In [15]:
data = read_data(datapath)
data = assign_color(data, 'catName')
pgs = unique_cats(data, 'catName')
single_dwg_data = unhex(data, 256)



In [16]:
pgs

['000 GSF',
 '08.43 Storefront',
 '08 Louvers',
 '04.10 CMU Masonry',
 '02.85 Fencing',
 '12.93 Site Furnishings',
 '09.65 Resilient Flooring',
 '02.27 Pavement Marking',
 '09.68 Carpeting',
 '08 Windows',
 '09.2 GWB Walls',
 '07 Insulation',
 '09.2 Partitions Wood',
 '09 Tile',
 '09.62 Specialty Flooring',
 '08.81 Glass Glazing',
 '000 Unit Counts',
 '07 Roofing',
 '09.30 Tiling',
 '12.36 Countertops',
 '000 Room Areas',
 '000 Unit Room Door Counts',
 '08 - Doors',
 '07 Cladding',
 '05.52 Metal Railings',
 '09.51 Acoustical Ceilings',
 '09.2 GWB Ceilings',
 '09.2 GWB Soffits']

In [10]:
#Select 
#bs.JobName, bc.Name, bc.Quantity1, bc.Quantity2, bc.Quantity3, bc.ColorLine,
#pg.Name as pagename, pg.Scale, pg.Width, pg.Height, pg.ScaleFactor1, pg.ScaleFactor2, pg.UID,
#tf.Position,
#bs.BidProjectUID,  tf.Quantity

#--tf1.Position as p1, tf2.Position as p2

#From BidTakeoffs as tf
#Join BidPages as pg On tf.BidPageUID = pg.UID
#--Full Outer Join BidTakeoffs as tf1 On ls.BidTakeoffFromUID = tf1.UID
#--Full Outer Join BidTakeoffs as tf2 On ls.BidTakeoffToUID = tf2.UID
#Join Bids as bs On bs.UID = pg.BidUID
#Left Join BidConditions as bc On tf.BidConditionUID = bc.UID

#--Join BidTypAreas as bt On bc.bid
#Where pg.UID = 82
#10262 aloft San Fran   Lobby Building Perimeter    SFO Aloft SD Dwgs 2010_07_12- PART_1.pdf (3) 

dpi = 216
bit_depth = 1

print(42 * 30 * dpi)
print(42 * dpi)

272160
9072
